In [1]:
import sys

sys.path.insert(0, "../")

Azure = True

import json
from torch.utils.data import DataLoader

from dataLoader.dataLoaderMLM import CoercionRiskDataset, process_data_CoercionRisk
from dataLoader.build_vocab import load_vocab, build_vocab

In [2]:
if Azure:
    file_config = {
        "vocab": "../dataloader/vocab.txt",  # vocabulary idx2token, token2idx
        "data_train": "../../EHR_data/data/pre_train_training_set.json",  # formated data
        "data_val": "../../EHR_data/CoercionData_val.json",  # formated data
        "model_path": "model/model1/",  # where to save model
        "model_name": "test",  # model name
        "file_name": "log",  # log path
        "use_cuda": True,
        "device$": "cuda:0",
    }
else:
    file_config = {
        "vocab": "/Users/mikkelsinkjaer/Library/Mobile Documents/com~apple~CloudDocs/transEHR/Code/transformerEHR/data/vocab.txt",  # vocabulary idx2token, token2idx
        "data_train": "/Users/mikkelsinkjaer/Library/Mobile Documents/com~apple~CloudDocs/transEHR/Code/transformerEHR/data/syntheticData.json",
        "data_val": "/Users/mikkelsinkjaer/Library/Mobile Documents/com~apple~CloudDocs/transEHR/Code/transformerEHR/data/syntheticData_val.json",
        "model_path": "model/model1/",  # where to save model
        "model_name": "test",  # model name
        "file_name": "log.txt",  # log path
        "use_cuda": False,
        "device": "cpu",
    }

In [3]:
with open(file_config["data_train"]) as f:
    data_json = json.load(f)
print(next(iter(data_json.items())))
vocab_list, word_to_idx = build_vocab(data_json, Azure=Azure)

('1513125', {'Sex': 'Mand', 'BirthDate': '2000-03-27', 'DeathDate': None, 'Events': [{'EncounterKey': 39462023, 'Time': '2018-03-07T00:53:00', 'Type': 'Behandlingskontakt'}, {'EncounterKey': 46510167, 'Time': '2018-09-12T08:30:00', 'Type': 'Aftale'}, {'EncounterKey': 53935369, 'Time': '2018-10-19T11:30:00', 'Type': 'Ambulant besøg'}, {'EncounterKey': 56159318, 'Time': '2018-12-07T12:00:00', 'Type': 'Aftale'}, {'EncounterKey': 55201214, 'Time': '2018-11-06T00:00:00', 'Type': 'Behandlingskontakt'}, {'EncounterKey': 53983470, 'Time': '2018-10-31T11:30:00', 'Type': 'Ambulant besøg'}, {'EncounterKey': 55008256, 'Time': '2018-11-03T00:00:00', 'Type': 'Behandlingskontakt'}, {'EncounterKey': 40522115, 'Time': '2018-06-27T08:15:00', 'Type': 'Aftale'}, {'EncounterKey': 55921027, 'Time': '2018-11-20T14:30:00', 'Type': 'Ambulant besøg'}, {'EncounterKey': 71252158, 'Time': '2019-07-10T16:50:00', 'Type': 'Aftale'}, {'EncounterKey': 34275256, 'Time': '2018-01-05T10:00:00', 'Type': 'Aftale'}, {'Encoun

Vocab: 100%|██████████| 72092/72092 [00:01<00:00, 44381.95it/s]


In [4]:
# Data loader
data = process_data_CoercionRisk(
    data_json, vocab_list, word_to_idx, mask_prob=0.20, Azure=Azure
)
sample_data = CoercionRiskDataset(data)
sample = next(iter(sample_data))

  1%|▏         | 1001/72092 [00:09<11:47, 100.53it/s]


In [5]:
import itertools

# sample = next(iter(data.items()))

# Replace 'N' with the desired item number you want to access (in this case, 10)
item_number = 10100

# Using itertools.islice to get the Nth item (key-value pair) from the dictionary
# sample = next(itertools.islice(data.items(), item_number))

count  = 0
for sample in data.items():
    index = sample[1]['sample_index']
    if sample[1]['classification_labels'] == 1:
        print('date',sample[1]['dates'][index-5:index+5])
        print('age',sample[1]['age'][index-5:index+5])
        print('codes',sample[1]['codes'][index-5:index+5])
        print('position',sample[1]['position'][index-5:index+5])
        print('segment',sample[1]['segment'][index-5:index+5])
        print('classification_labels',sample[1]['classification_labels'])
        print('original_code_sequence',sample[1]['original_code_sequence'][index-5:index+5])
        print('sample_index',sample[1]['sample_index'])
        print('input_sequence',sample[1]['input_sequence'][index-5:index+5])
        print('attention_mask',sample[1]['attention_mask'][index-5:index+5])
        break
    count +=1




date [43591, 43591, 43591, 43591, 0, 0, 0, 0, 0, 0]
age [69.33333333333333, 69.33333333333333, 69.33333333333333, 69.33333333333333, 69.33333333333333, 69.58333333333333, 69.58333333333333, 69.58333333333333, 70.58333333333333, 69.25]
codes ['<SEP>', 'DF20', 'DF20', 'DF2', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
position [19, 20, 20, 20, 20, 20, 20, 20, 20, 20]
segment [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
classification_labels 1
original_code_sequence ['<SEP>', 'DF20', 'DF20', 'DF2', 'DF2', 'psych_admission', 'coercion_Beroligende medicin', 'coercion_Beroligende medicin', 'Behandlingskontakt', '0 Indlagt']
sample_index 63
input_sequence [2, 64, 64, 53, 3, 3, 3, 3, 3, 3]
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
